# Pytorch

* [Pytorch tutorials](https://pytorch.org/tutorials/)
* [Pytorch Documentation](https://pytorch.org/docs/stable/index.html)

一些事实：

* `pytorch` 使用`torch.Tensor`作为矩阵类型。`torch.Tensor` = 矩阵数据 + 设备信息 + 梯度信息
* `pytorch` = 自动微分框架 + 典型网络层 + 数据加载与并行 + 训练工具 + 部署 + 插件

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

In [ ]:
import torch
import pdir
import numpy as np
import matplotlib.pyplot as plt

Pytorch Tensor的使用方式基本类似于numpy

In [ ]:
x = torch.rand(4, 4)
y = np.random.rand(4, 4)

In [ ]:
type(x)

In [ ]:
type(y)

`pytorch`与`numpy`并不互通

In [ ]:
x@y

In [ ]:
y@x

In [ ]:
x.numpy() @ y

In [ ]:
x @ torch.Tensor(y)

# Tensor

`tensor(data, dtype=None, device=None, requires_grad=False, pin_memory=False) -> Tensor`

### Conversions vs Constructions

`torch.Tensor`只在必要的时候进行数据构造

In [ ]:
X = torch.rand(4, 4)
Y = torch.Tensor(X)
X, Y

In [ ]:
Y[1] = 0
X, Y # X is changed as well

必要的时候：从numpy.array构造

In [ ]:
X = np.random.rand(4, 4)
Y = torch.Tensor(X)
X, Y

In [ ]:
Y[1] = 0
X, Y # X isn't changed

而`torch.tensor`一定会进行数据构造

In [ ]:
X = torch.rand(4, 4)
Y = torch.tensor(X)
X, Y

In [ ]:
Y[1] = 0
X, Y # X is not changed

In [ ]:
X = torch.rand(4, 4)
Y = X.clone().detach()
X, Y

In [ ]:
Y[1] = 0
X, Y # X is not changed

# 自动微分框架

自动微分框架是指：根据前向传播的计算图自动推断反向传播的计算图，并给出导数

In [ ]:
X = torch.rand(4, 4, requires_grad=True)
Y = torch.rand(4, 4, requires_grad=True)

In [ ]:
print(X.grad)
print(Y.grad)

`backward`将所有与loss有关的Tensor的梯度全部更新

In [ ]:
loss = torch.sum(torch.pow(X, 2)-Y)
loss.backward()

In [ ]:
print(X.grad)
print(Y.grad)

梯度需要手动清除

In [ ]:
X.grad.data.zero_()
Y.grad.data.zero_()

### 作业5.1：利用Pytorch的自动微分框架来重新实现作业一 (12.16之前提交)

# 典型网络层

Pytorch的网络层都在`torch.nn`中

基本的使用方式是：

1. 查看`torch.nn`提供了什么现成的（判断是不是需要自己手写）
2. 如果有现成的话，查看感兴趣的网络层的帮助文档

In [ ]:
pdir(torch.nn)

In [ ]:
from sklearn import datasets
import math
def load_boston(ratio=0.8):
    X, Y = datasets.load_boston(True)
    Y.shape = -1, 1
    
    # normalization
    X = X/80
    Y = Y/(np.max(Y) - np.min(Y))
    
    num_samples = len(Y)
    num_train = math.ceil(num_samples * ratio)
    
    # 随机打乱数据
    idx = np.random.permutation(np.arange(num_samples))
    traindata = X[idx[:num_train]], Y[idx[:num_train]]
    validdata = X[idx[num_train:]], Y[idx[num_train:]]
    
    return traindata, validdata

In [ ]:
(X_train, Y_train), (X_valid, Y_valid) = load_boston()
X_train, Y_train = torch.Tensor(X_train), torch.Tensor(Y_train)
X_valid, Y_valid = torch.Tensor(X_valid), torch.Tensor(Y_valid)


In [ ]:
model = torch.nn.Sequential(
    torch.nn.Linear(13, 50),
    torch.nn.Sigmoid(),
    torch.nn.Linear(50, 1)
)
model

In [ ]:
loss = torch.nn.MSELoss()

In [ ]:
opt = torch.optim.SGD(model.parameters(), lr=1e-3)

In [ ]:
valid_losses = []
train_losses = []
for i in range(1000):
    X, Y = X_train, Y_train
    
    # 梯度清零
    opt.zero_grad()
    
    # 前向传播 -- 得到反向传播所需要的中间结果
    l = loss(model(X), Y)
    
    # 反向传播 -- 计算梯度
    l.backward()
    
    # 更新权重
    opt.step()
    
    with torch.no_grad():
        cur_train_loss = l
        cur_valid_loss = loss(Y_valid, model(X_valid))
        valid_losses.append(cur_valid_loss) 
        train_losses.append(cur_train_loss)

        if i%100 == 0:
            print(f"Iter {i}: loss {cur_train_loss:.4f}, valid loss {cur_valid_loss:.4f}")

In [ ]:
plt.plot(train_losses)
plt.plot(valid_losses)
plt.legend(["train loss", "validation loss"])

# 实现新的网络层

继承 `torch.nn.Module` 即可

In [ ]:
class FCLayer(torch.nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        
        # nn.Parameter用来告诉optimizer该权重是需要更新的 -- model.parameters()
        self.W = torch.nn.Parameter(torch.rand(in_features, out_features))
        self.b = 1
    
    def forward(self, X):
        return X @ self.W + 1

In [ ]:
model = torch.nn.Sequential(
    FCLayer(13, 50),
    torch.nn.Sigmoid(),
    FCLayer(50, 1)
)
model(X_train).shape

In [ ]:
model

In [ ]:
class MyModel(torch.nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        
        self.fc1 = torch.nn.Linear(13, 50)
        self.activation = torch.nn.Sigmoid()
        self.fc2 = torch.nn.Linear(50, 1)
        
    def forward(self, X):
        return self.fc2(self.activation(self.fc1(X)))

In [ ]:
import torchvision

In [ ]:
model

In [ ]:
model = MyModel(13, 1)
model(X_train).shape

### 作业5.2：利用Pytorch的`torch.nn.Module`来重新实现小测验 (12.16之前提交)

* 网络层自己用torch.nn.Module实现
* 优化器和loss使用现成的即可

# 数据读取

`torch.utils.data.Dataset`, `torch.utils.data.Dataloader` 用来处理大规模数据

# 训练记录

`Tensorboard`

![](./quickstart_gradient_tape.png)

# 插件

`torchvision` 提供了一些更计算机视觉相关的工具

# 部署

`onnx`(Open Neural Network Exchange Format)模块提供了模型在不同框架下的转换及部署

https://pytorch.org/docs/stable/index.html